# Modelowanie liczby roszczeń

### Generowanie niejednorodnego procesu Poissona metodą przerzedzania

In [1]:
import numpy as np
import pandas as pd

Oznaczenia
<br/>
n - liczba zdarzeń w każdym roku
<br/>
N - skumulowana licza zdarzeń od początku

In [2]:
def nhpp(lambda_func,lambda_func_params, t_max):
    """
    Funkcja generuje niejednorodny proces Poissona metodą przerzerzedzania.

    Argumenty:
        lambda_func: funkcja
            Funkcja intensywności
        lambda_func_params: tab
            Tablica zawierająca parametry funkcji intensywności
        t_max: float
            Maksymalny czas symulacji
        lambda_max: float
            Ograniczenie górne funkcji lambda_func
    Zwraca:
        numpy.ndarray
            Tablica czasów.
    """

    S = 0
    N = 0
    S_tab = []
    lambda_max = lambda_func(0,lambda_func_params)[1]
    
    while S < t_max:
        U = np.random.uniform(0,1)
        S=S+(-1/lambda_max)*np.log(U)
        if S > t_max:
            break
        U = np.random.uniform(0,1)
        if U <= lambda_func(S,lambda_func_params)[0]/lambda_max:
            N = N + 1
            S_tab.append(S)

    return N, S_tab

In [3]:
def sin_int_fun(t,lambda_func_params):
    """
    Sinusoidalna funkcja intensywności.

    Argumenty:
        t: float
            Funkcja intensywności
        lambda_func_params: tab
            Tablica zawierająca parametry funkcji intensywności.
            W tym przypadku listę zawierającą: [lambda_0, lambda_1, T, phi]
    Zwraca:
        float
            Wartość funkcji intensywności.
        float
            Górne ograniczenie funkcji.
            W tym przypadku lambda_0+lambda_1.    
    """
    return lambda_func_params[0]+lambda_func_params[1]*np.sin(2*np.pi*t/lambda_func_params[2]+lambda_func_params[3]), lambda_func_params[0]+lambda_func_params[1]

In [4]:
def exp_int_fun(t,lambda_func_params):
    """
    Wykładnicza funkcja intensywności.

    Argumenty:
        t: float
            Funkcja intensywności
        lambda_func_params: tab
            Tablica zawierająca parametry funkcji intensywności. 
            W tym przypadku listę zawierającą: [lambda_0, lambda]
    Zwraca:
        float
            Wartość funkcji intensywności.
        float
            Górne ograniczenie funkcji.
            W tym przypadku lambda_0/lambda.
    """
    return lambda_func_params[0]*np.exp(-lambda_func_params[1]*t), lambda_func_params[0]*lambda_func_params[1]

In [5]:
nhpp(sin_int_fun,[1,3,1,0],10)

(19,
 [0.08476798824574257,
  1.0829428951571487,
  2.133882439130639,
  2.290059868588918,
  3.1713135458955946,
  3.205405245720097,
  4.200630809340486,
  4.335652335369483,
  4.4419694243786205,
  5.114303232877399,
  5.307173673377175,
  6.2169582722376635,
  6.320841130104374,
  7.095540927159271,
  7.311501147331578,
  8.074579585004779,
  8.158926144638983,
  9.186366334332348,
  9.269841480425647])

In [6]:
nhpp(exp_int_fun,[2,1/10],100)

(7,
 [3.5297101261192982,
  9.790063011528808,
  13.039244137267655,
  20.475916639147812,
  25.77141937935908,
  37.78350950618513,
  89.57232474549063])

Empiryczny proces $N_t$.

In [7]:
data = pd.read_csv("https://github.com/ndzadz/mgr/blob/main/data_us_1990_2022.csv?raw=true",encoding="latin1",sep=";",header=0)
p95 = np.percentile(data[data["Insured Damages, Adjusted (\'000 US$)"].isna()==0]["Insured Damages, Adjusted (\'000 US$)"],95)
data = data[(data["Insured Damages, Adjusted (\'000 US$)"].isna()==0)&(data["Insured Damages, Adjusted (\'000 US$)"]<p95)]
data = data.rename(columns={'ï»¿Dis No':'Dis No'})

In [148]:
n = pd.Series(data.groupby('Year').count()['Dis No'])
n.index = range(1,len(n)+1)
n

1     17
2     28
3     29
4     25
5     11
6     18
7     12
8     26
9     26
10    16
11     7
12    10
13    13
14    13
15    13
16     9
17    12
18    22
19    13
20    10
21     9
22    16
23    20
24    20
25    15
26    25
27    21
28    32
29     4
30    10
31     8
32     1
33     2
Name: Dis No, dtype: int64

In [149]:
N = np.cumsum(n)
N

1      17
2      45
3      74
4      99
5     110
6     128
7     140
8     166
9     192
10    208
11    215
12    225
13    238
14    251
15    264
16    273
17    285
18    307
19    320
20    330
21    339
22    355
23    375
24    395
25    410
26    435
27    456
28    488
29    492
30    502
31    510
32    511
33    513
Name: Dis No, dtype: int64

In [150]:
def MSE(n, n_pred):
    k = len(n)
    return np.sum(np.array((n-n_pred)**2))/k

In [151]:
def MAE(n, n_pred):
    k = len(n)
    return np.sum(np.abs((n-n_pred)))/k

Metoda najmniejszej sumy kwadratów

In [152]:
from scipy.optimize import minimize

Sinusoidalna funkcja intensywności

In [153]:
def sum_of_squares_sin(lambda_func_params):
    t = np.array(range(len(n)))
    return np.sum(np.array((n-sin_int_fun(t, lambda_func_params)[0])**2))

In [154]:
sum_of_squares_sin([1,3,1,0])

9014.000000000011

In [155]:
bnds = ((0, None), (0, None), (0, None), (0, None)) # ograniczenia na wartości parametrów
params = [1.0, 1.0, 1.0, 1.0] # Przyjmujemy początkowe wartości parametrów
result = minimize(sum_of_squares_sin, params, bounds=bnds, method='L-BFGS-B')
print(result)


      fun: 1375.1579339596738
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 4.54747313e-05, -2.27373677e-05, -2.51702660e-02,  4.54747313e-04])
  message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 255
      nit: 42
     njev: 51
   status: 0
  success: True
        x: array([14.86573535,  6.34683826,  1.05347731,  8.96354863])


Symulacja procesu z sinusoidalną funkcją intensywności dla wyliczonych parametrów

In [160]:
n_pred = nhpp(sin_int_fun,result.x,33)
print(n)

1     17
2     28
3     29
4     25
5     11
6     18
7     12
8     26
9     26
10    16
11     7
12    10
13    13
14    13
15    13
16     9
17    12
18    22
19    13
20    10
21     9
22    16
23    20
24    20
25    15
26    25
27    21
28    32
29     4
30    10
31     8
32     1
33     2
Name: Dis No, dtype: int64


In [161]:
unique, counts = np.unique(np.ceil(n_pred[1]), return_counts=True)
values = pd.DataFrame({'Year':unique, 'n':counts})
index_df = pd.DataFrame({'Year':np.array(range(1,34))})
merged_table = pd.merge(index_df, values, on='Year', how='left').fillna(0).set_index('Year')
n_pred = pd.Series(merged_table['n'])
N_pred = np.cumsum(n_pred)
print(n_pred)

Year
1     15
2     17
3     22
4     15
5     15
6     17
7     20
8     15
9      8
10    11
11    15
12    19
13    22
14    15
15    17
16    11
17    12
18    12
19    10
20    11
21    17
22    14
23    19
24    10
25    12
26    10
27    22
28    11
29    15
30    16
31    17
32    12
33    17
Name: n, dtype: int64


In [126]:
print("MSE: ", MSE(n,n_pred), "MAE: ", MAE(n,n_pred))

MSE:  91.03030303030303 MAE:  8.06060606060606


Wykładnicza funkcja intensywności

In [127]:
def sum_of_squares_exp(lambda_func_params):
    t = np.array(range(len(n)))
    return np.sum(np.array((n-exp_int_fun(t, lambda_func_params)[0])**2))

In [128]:
sum_of_squares_exp([1,3])

9971.064324374476

In [129]:
bnds = ((0, None), (0, None)) # ograniczenia na wartości parametrów
params = [1.0, 1.0] # Przyjmujemy początkowe wartości parametrów
result = minimize(sum_of_squares_exp, params, bounds=bnds, method='L-BFGS-B')
print(result)


      fun: 1728.9810271263777
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>
      jac: array([-6.82120970e-05,  3.15367288e-02])
  message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 81
      nit: 11
     njev: 27
   status: 0
  success: True
        x: array([2.12313005e+01, 2.07091572e-02])


Symulacja procesu z sinusoidalną funkcją intensywności dla wyliczonych parametrów

In [134]:
n_pred = nhpp(exp_int_fun,result.x,33)
print(n_pred)

(10, [8.984835209980888, 11.78057936910135, 11.924656398677355, 12.079714722174632, 15.384835002035674, 16.815081009281883, 21.41136765840557, 24.078638837688565, 29.105988404412884, 32.983114107055606])


In [138]:
unique, counts = np.unique(np.ceil(n_pred[1]), return_counts=True)
values = pd.DataFrame({'Year':unique, 'n':counts})
index_df = pd.DataFrame({'Year':np.array(range(1,34))})
merged_table = pd.merge(index_df, values, on='Year', how='left').fillna(0).set_index('Year')
n_pred = pd.Series(merged_table['n'])
N_pred = np.cumsum(n_pred)
print(n_pred)

Year
1     0.0
2     0.0
3     0.0
4     0.0
5     0.0
6     0.0
7     0.0
8     0.0
9     0.0
10    0.0
11    0.0
12    0.0
13    0.0
14    0.0
15    0.0
16    0.0
17    0.0
18    0.0
19    0.0
20    0.0
21    0.0
22    0.0
23    0.0
24    0.0
25    0.0
26    0.0
27    0.0
28    0.0
29    0.0
30    0.0
31    0.0
32    0.0
33    0.0
Name: n, dtype: float64


In [139]:
print("MSE: ", MSE(n,n_pred), "MAE: ", MAE(n,n_pred))

MSE:  303.24242424242425 MAE:  15.545454545454545
